# Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN data set.

# Import statements
Following packages are necessary:

In [ ]:
import sys
import random
import tensorflow as tf
import numpy as np
from endolas import utils
from endolas import ccall
from endolas import UNet
from endolas import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from endolas import LASTENSequence

# Checks
The version of tensorflow as well as the GPU support are checked.

In [ ]:
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.list_physical_devices('GPU')

# Seeding
Seeds are set to ensure reproducible training.

In [ ]:
tf.keras.backend.clear_session()

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Data
Import training images

In [ ]:
path_train = r'Data/LASTEN/train'
path_validation = r'Data/LASTEN/validation'

width = 384
height = 384

train_gen = LASTENSequence(path_train,
                           batch_size=1,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=1,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [ ]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

# Training Preparation
Prepare the settings for training the model.

In [ ]:
save_path =  r'Data/LASTEN/results'

logger = CSVLogger(save_path + "/log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5", period=10)
checker_best = ModelCheckpoint(save_path + "/best_weights.hdf5", save_best_only=True)
timelogger = ccall.TimeHistory(save_path)

callbacks = [logger, checker, checker_best, timelogger]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a keras Sequence.

In [ ]:
model.fit(train_gen,
          epochs=100,
          callbacks=callbacks,
          validation_data=validation_gen,
          validation_freq=1,)